In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Library Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

### Load the datasets and verify the data load

In [ ]:
authors = pd.read_csv("../input/nips-papers-1987-2019-updated/authors.csv")

authors.head()

In [ ]:
# A quick look on the data structure and missing values

authors.info()

authors.isna().sum()

In [ ]:
# Load the papers dataset

papers = pd.read_csv("../input/nips-papers-1987-2019-updated/papers.csv")

papers.head()

In [ ]:
# A quick look on the papers data structure and missing values by columns

papers.info()

papers.isna().sum()

we would perform the missing value imputations, by putting in the correspoinding title text for the missing values in both abstract and full_text columns

In [ ]:
# perform the missing imputations on both full_text and abstract

papers['full_text'] = np.where(papers['full_text'].isna(), papers['title'], papers['full_text'])
papers['abstract'] = np.where(papers['abstract'].isna(), papers['title'], papers['abstract'])

In [ ]:
# Verify the imputations
papers.isna().sum()

### Preprocessing

#### Task: Use TF-IDF Vectorization to create a vectorized document term matrix. We may want to explore the max_df and min_df parameters.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
dtm = tfidf.fit_transform(papers['title'])

dtm

### Latent Drichilet Allocation

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=7, random_state=101)

# This can take a while, we are dealing with large number of documents here
LDA.fit(dtm)

In [ ]:
# Check the shape of the numpy array - This matches with 10 components which we wanted and columns equal to the sparse matrix
LDA.components_.shape

In [ ]:
# Now we will attach the topic numbers to the original articles

# In order to do this, we would need to apply the transform method on our LDA model and DTM to associate the topic to each
# article
topic_results = LDA.transform(dtm)

# Now we will check the shape
topic_results.shape

In [ ]:
# If we check the first element, we will get the probabilities of each of the articles belonging to one of the 7 topics
topic_results[0]

There is a 69% probability, that it should belong to the 5th topic

In [ ]:
# Associating topic to each of the documents

papers['Topic'] = topic_results.argmax(axis=1)

papers.head(10)

#### Now we will setup a loop which will print out the top 10 words with highest probabilities for all the ten topics

In [ ]:
for index, topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    list_keywords = [tfidf.get_feature_names()[index] for index in topic.argsort()[-10:]]
    print(list_keywords)
    print("\n")

> #### A basic visualization on number of documents by topics

In [ ]:

sns.countplot(x = 'Topic',
              data = papers,
              order = papers['Topic'].value_counts().index)

papers['Topic'].value_counts().sort_values(ascending=False)